# Démonstration : RSA Encryption & Signatures

**Objectifs** :
- Implémenter RSA (génération de clés, chiffrement, signatures)
- Comprendre le problème de factorisation
- Démontrer les attaques sur RSA textbook
- Utiliser RSA-OAEP (standard sécurisé)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.backends import default_backend
import secrets
import time

## 1. RSA : Vue d'ensemble

**Basé sur** : Difficulté de factorisation des grands entiers

**Génération de clés** :
1. Choisir deux grands premiers $p, q$ (1024 bits chacun)
2. Calculer $n = p \cdot q$ (modulus, 2048 bits)
3. Calculer $\phi(n) = (p-1)(q-1)$
4. Choisir $e$ tel que $\gcd(e, \phi(n)) = 1$ (souvent $e = 65537$)
5. Calculer $d = e^{-1} \mod \phi(n)$
6. Clé publique : $(n, e)$
7. Clé privée : $(n, d)$ (ou $(p, q, d)$ avec CRT)

**Chiffrement** : $c = m^e \mod n$

**Déchiffrement** : $m = c^d \mod n$

**Pourquoi ça marche** : $m^{ed} \equiv m \pmod{n}$ (Théorème d'Euler)

## 2. Génération de Clés RSA

In [ ]:
def generate_rsa_keypair():
    """
    Génère une paire de clés RSA-2048.
    """
    print("=" * 70)
    print("GÉNÉRATION DE CLÉS RSA-2048")
    print("=" * 70)
    
    print("\n⏳ Génération en cours (recherche de nombres premiers)...")
    start = time.perf_counter()
    
    # Génération de clé RSA-2048
    private_key = rsa.generate_private_key(
        public_exponent=65537,  # e = 65537 (standard)
        key_size=2048,
        backend=default_backend()
    )
    
    elapsed = time.perf_counter() - start
    print(f"✅ Génération terminée en {elapsed:.2f}s")
    
    # Extraction des paramètres
    public_key = private_key.public_key()
    
    private_numbers = private_key.private_numbers()
    public_numbers = private_key.public_key().public_numbers()
    
    p = private_numbers.p
    q = private_numbers.q
    n = public_numbers.n
    e = public_numbers.e
    d = private_numbers.d
    
    print(f"\n📋 Paramètres RSA :")
    print(f"   p (premier 1) : {p.bit_length()} bits")
    print(f"   q (premier 2) : {q.bit_length()} bits")
    print(f"   n = p × q     : {n.bit_length()} bits")
    print(f"   e (exposant public) : {e}")
    print(f"   d (exposant privé)  : {d.bit_length()} bits")
    
    # Vérifications
    phi_n = (p - 1) * (q - 1)
    print(f"\n🔍 Vérifications :")
    print(f"   p × q = n ? {p * q == n} ✅")
    print(f"   e × d ≡ 1 (mod φ(n)) ? {(e * d) % phi_n == 1} ✅")
    
    return private_key, public_key

private_key, public_key = generate_rsa_keypair()

## 3. RSA Textbook (INSÉCURISÉ !)

**Définition** : $c = m^e \mod n$ (chiffrement direct)

**Problèmes** :
- ❌ Déterministe (pas CPA-sécurisé)
- ❌ Homomorphe multiplicatif (malléable)
- ❌ Vulnérable aux petits messages

**Démonstration** :

In [ ]:
def rsa_textbook_demo():
    """
    Démonstration RSA textbook (éducatif uniquement).
    
    ⚠️ NE JAMAIS utiliser en production !
    """
    print("\n" + "=" * 70)
    print("RSA TEXTBOOK (INSÉCURISÉ - démo seulement)")
    print("=" * 70)
    
    # Extraction paramètres
    n = public_key.public_numbers().n
    e = public_key.public_numbers().e
    d = private_key.private_numbers().d
    
    # Message
    message = b"Hello, RSA!"
    message_int = int.from_bytes(message, 'big')
    
    print(f"\n📝 Message :")
    print(f"   Texte  : {message}")
    print(f"   Entier : {message_int}")
    
    # Chiffrement : c = m^e mod n
    ciphertext_int = pow(message_int, e, n)
    
    print(f"\n🔐 Chiffrement (c = m^e mod n) :")
    print(f"   Ciphertext : {ciphertext_int}")
    print(f"   Bits       : {ciphertext_int.bit_length()}")
    
    # Déchiffrement : m = c^d mod n
    decrypted_int = pow(ciphertext_int, d, n)
    byte_length = (decrypted_int.bit_length() + 7) // 8
    decrypted = decrypted_int.to_bytes(byte_length, 'big')
    
    print(f"\n🔓 Déchiffrement (m = c^d mod n) :")
    print(f"   Message : {decrypted}")
    print(f"   Correct : {decrypted == message} ✅")
    
    # Problème 1 : Déterministe
    print(f"\n❌ Problème 1 : Déterministe (pas CPA-sécurisé)")
    ciphertext_int_2 = pow(message_int, e, n)
    print(f"   Même message → même ciphertext : {ciphertext_int == ciphertext_int_2}")
    print(f"   → Attaquant peut détecter messages répétés !")
    
    # Problème 2 : Homomorphisme
    print(f"\n❌ Problème 2 : Homomorphisme multiplicatif")
    m1 = 100
    m2 = 5
    c1 = pow(m1, e, n)
    c2 = pow(m2, e, n)
    c_prod = (c1 * c2) % n
    m_prod = pow(c_prod, d, n)
    print(f"   E({m1}) × E({m2}) = E({m1 * m2})")
    print(f"   Vérifié : {m_prod == (m1 * m2) % n} ✅")
    print(f"   → Malléabilité : Eve peut modifier les ciphertexts !")
    
    print(f"\n⚠️  RSA TEXTBOOK EST DANGEUREUX !")
    print(f"✅ Solution : Utiliser RSA-OAEP (avec padding aléatoire)")

rsa_textbook_demo()

## 4. RSA-OAEP (Standard Sécurisé)

**OAEP** : Optimal Asymmetric Encryption Padding

**Principe** :
1. Ajouter un padding aléatoire au message
2. Appliquer transformations avec fonctions de hachage
3. Chiffrer avec RSA textbook

**Résultat** :
- ✅ CPA-sécurisé (randomisé)
- ✅ Résistant aux attaques (chosen-ciphertext)
- ✅ Standard recommandé (PKCS#1 v2.0)

In [ ]:
def rsa_oaep_demo():
    """
    Démonstration RSA-OAEP (standard sécurisé).
    """
    print("\n" + "=" * 70)
    print("RSA-OAEP (Standard Sécurisé)")
    print("=" * 70)
    
    message = b"Confidential message encrypted with RSA-OAEP"
    
    print(f"\n📝 Message : {message}")
    print(f"   Taille : {len(message)} bytes")
    
    # Chiffrement RSA-OAEP
    print(f"\n🔐 Chiffrement avec RSA-OAEP (SHA-256) :")
    ciphertext = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    print(f"   Ciphertext (hex) : {ciphertext.hex()[:64]}...")
    print(f"   Taille           : {len(ciphertext)} bytes")
    
    # Déchiffrement
    print(f"\n🔓 Déchiffrement :")
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    print(f"   Message déchiffré : {plaintext}")
    print(f"   Correct : {plaintext == message} ✅")
    
    # Test : Randomisation
    print(f"\n✅ Vérification : Randomisation (CPA-sécurisé)")
    ciphertext2 = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    print(f"   Même message, 2 chiffrements :")
    print(f"   CT1 : {ciphertext.hex()[:32]}...")
    print(f"   CT2 : {ciphertext2.hex()[:32]}...")
    print(f"   Différents : {ciphertext != ciphertext2} ✅")
    print(f"   → Impossible de détecter messages identiques")
    
    # Limitation de taille
    print(f"\n📏 Limitation : Taille maximale du message")
    max_size = (2048 // 8) - 2 * (256 // 8) - 2  # (key_size - 2*hash_size - 2)
    print(f"   RSA-2048 avec SHA-256 : {max_size} bytes max")
    print(f"   Pour messages plus longs : Chiffrement hybride (RSA + AES)")

rsa_oaep_demo()

## 5. RSA Signatures

**Principe** : "Déchiffrer" avec la clé privée pour prouver l'identité

**Signature** : $s = H(m)^d \mod n$

**Vérification** : $s^e \stackrel{?}{=} H(m) \pmod{n}$

**Standard** : RSA-PSS (Probabilistic Signature Scheme)

In [ ]:
def rsa_signature_demo():
    """
    Démonstration de signatures RSA-PSS.
    """
    print("\n" + "=" * 70)
    print("SIGNATURES RSA-PSS")
    print("=" * 70)
    
    message = b"I, Alice, transfer $1000 to Bob."
    
    print(f"\n📝 Message à signer : {message}")
    
    # Signature (Alice)
    print(f"\n✍️  Alice signe avec sa clé privée :")
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    print(f"   Signature (hex) : {signature.hex()[:64]}...")
    print(f"   Taille          : {len(signature)} bytes")
    
    # Vérification (Bob)
    print(f"\n✅ Bob vérifie avec la clé publique d'Alice :")
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print(f"   Signature VALIDE ✅")
        print(f"   → Message authentifié (vient bien d'Alice)")
        print(f"   → Message intègre (pas modifié)")
    except Exception as e:
        print(f"   Signature INVALIDE ❌")
        print(f"   Erreur : {e}")
    
    # Test : Modification du message
    print(f"\n🔍 Test : Modification du message")
    modified_message = b"I, Alice, transfer $9999 to Eve."
    
    print(f"   Message modifié : {modified_message}")
    
    try:
        public_key.verify(
            signature,
            modified_message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print(f"   ❌ ERREUR : Signature valide sur message modifié !")
    except Exception:
        print(f"   Signature INVALIDE ✅")
        print(f"   → Modification DÉTECTÉE et REJETÉE")
    
    # Test : Modification de la signature
    print(f"\n🔍 Test : Modification de la signature")
    corrupted_signature = bytearray(signature)
    corrupted_signature[0] ^= 0xFF
    
    try:
        public_key.verify(
            bytes(corrupted_signature),
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print(f"   ❌ ERREUR : Signature corrompue acceptée !")
    except Exception:
        print(f"   Signature INVALIDE ✅")
        print(f"   → Corruption DÉTECTÉE et REJETÉE")

rsa_signature_demo()

## 6. Attaque : Petits Exposants

**Contexte** : Si $e$ est petit (ex: $e = 3$) et message court.

**Attaque** : $c = m^3 < n$ → calculer simplement $m = \sqrt[3]{c}$ (racine cubique réelle) !

In [ ]:
def small_exponent_attack_demo():
    """
    Démonstration d'attaque sur petit exposant.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Petit Exposant (e = 3)")
    print("=" * 70)
    
    print(f"\n⚠️  Scénario vulnérable :")
    print(f"   - Exposant public e = 3 (petit)")
    print(f"   - Message court : m < n^(1/3)")
    print(f"   - Pas de padding (RSA textbook)")
    
    print(f"\n💡 Si m^3 < n, alors c = m^3 (sans modulo !)")
    print(f"   → Attaquant peut calculer m = ∛c (racine cubique réelle)")
    
    print(f"\n✅ PROTECTIONS :")
    print(f"   1. Utiliser e = 65537 (standard)")
    print(f"   2. Toujours utiliser padding (OAEP)")
    print(f"   3. Messages > n^(1/e) grâce au padding")
    
    print(f"\n📊 Exemple avec RSA-2048 :")
    n_bits = 2048
    e = 65537
    safe_message_bits = n_bits // e
    print(f"   n : {n_bits} bits")
    print(f"   e : {e}")
    print(f"   Message sûr : < {safe_message_bits} bits (sans padding)")
    print(f"   Avec OAEP : padding force m > n^(1/e) → Sécurisé ✅")

small_exponent_attack_demo()

## 7. Performance : RSA vs ElGamal vs ECC

In [ ]:
def benchmark_public_key():
    """
    Compare les performances de RSA.
    """
    print("\n" + "=" * 70)
    print("BENCHMARK : Opérations RSA")
    print("=" * 70)
    
    message = b"Test message for benchmarking"
    iterations = 100
    
    print(f"\nItérations : {iterations}\n")
    
    # Chiffrement RSA-OAEP
    start = time.perf_counter()
    for _ in range(iterations):
        ciphertext = public_key.encrypt(
            message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
    time_encrypt = time.perf_counter() - start
    
    # Déchiffrement RSA-OAEP
    start = time.perf_counter()
    for _ in range(iterations):
        plaintext = private_key.decrypt(
            ciphertext,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
    time_decrypt = time.perf_counter() - start
    
    # Signature RSA-PSS
    start = time.perf_counter()
    for _ in range(iterations):
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
    time_sign = time.perf_counter() - start
    
    # Vérification signature
    start = time.perf_counter()
    for _ in range(iterations):
        try:
            public_key.verify(
                signature,
                message,
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
        except:
            pass
    time_verify = time.perf_counter() - start
    
    print(f"Opération          | Temps total | Par opération")
    print(f"-" * 60)
    print(f"Chiffrement (pub)  | {time_encrypt:>8.3f}s   | {time_encrypt/iterations*1000:>6.2f} ms")
    print(f"Déchiffrement (priv)| {time_decrypt:>8.3f}s   | {time_decrypt/iterations*1000:>6.2f} ms")
    print(f"Signature (priv)   | {time_sign:>8.3f}s   | {time_sign/iterations*1000:>6.2f} ms")
    print(f"Vérification (pub) | {time_verify:>8.3f}s   | {time_verify/iterations*1000:>6.2f} ms")
    
    print(f"\n📊 Observations :")
    print(f"   Opérations avec clé publique (chiffrement, vérif) : Rapides")
    print(f"   Opérations avec clé privée (déchiffrement, sign) : Lentes")
    print(f"   → Ratio déchiffrement/chiffrement : ~{time_decrypt/time_encrypt:.0f}x")
    
    print(f"\n💡 Optimisation : Chinese Remainder Theorem (CRT)")
    print(f"   Accélère déchiffrement et signature (~4x plus rapide)")

benchmark_public_key()

## 8. Chiffrement Hybride (RSA + AES)

**Problème** : RSA lent et limite la taille des messages.

**Solution** : Chiffrement hybride
1. Générer clé AES aléatoire $k$
2. Chiffrer message avec AES-GCM : $c_m = \text{AES-GCM}_k(m)$
3. Chiffrer clé AES avec RSA-OAEP : $c_k = \text{RSA-OAEP}(k)$
4. Envoyer : $(c_k, c_m)$

**Avantages** :
- ✅ Rapide (AES pour données)
- ✅ Sécurisé (RSA pour clé)
- ✅ Pas de limite de taille

**Utilisé dans** : TLS, PGP, S/MIME

In [ ]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

def hybrid_encryption_demo():
    """
    Démonstration de chiffrement hybride RSA + AES-GCM.
    """
    print("\n" + "=" * 70)
    print("CHIFFREMENT HYBRIDE : RSA + AES-GCM")
    print("=" * 70)
    
    # Message long
    message = b"This is a very long message that would be inefficient to encrypt directly with RSA. " * 10
    
    print(f"\n📝 Message : {len(message)} bytes")
    
    # Étape 1 : Générer clé AES
    aes_key = AESGCM.generate_key(bit_length=256)
    aesgcm = AESGCM(aes_key)
    
    print(f"\n1️⃣ Génération clé AES-256 : {aes_key.hex()[:32]}...")
    
    # Étape 2 : Chiffrer message avec AES-GCM
    nonce = secrets.token_bytes(12)
    ciphertext_aes = aesgcm.encrypt(nonce, message, None)
    
    print(f"\n2️⃣ Chiffrement message avec AES-GCM :")
    print(f"   Taille ciphertext : {len(ciphertext_aes)} bytes")
    
    # Étape 3 : Chiffrer clé AES avec RSA-OAEP
    ciphertext_rsa = public_key.encrypt(
        aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    print(f"\n3️⃣ Chiffrement clé AES avec RSA-OAEP :")
    print(f"   Taille ciphertext : {len(ciphertext_rsa)} bytes")
    
    print(f"\n📤 Transmission : (c_rsa, nonce, c_aes)")
    print(f"   Total : {len(ciphertext_rsa) + len(nonce) + len(ciphertext_aes)} bytes")
    
    # Déchiffrement
    print(f"\n🔓 Déchiffrement :")
    
    # Étape 1 : Déchiffrer clé AES avec RSA
    aes_key_decrypted = private_key.decrypt(
        ciphertext_rsa,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    print(f"   1️⃣ Clé AES déchiffrée avec RSA")
    
    # Étape 2 : Déchiffrer message avec AES-GCM
    aesgcm_decrypt = AESGCM(aes_key_decrypted)
    plaintext = aesgcm_decrypt.decrypt(nonce, ciphertext_aes, None)
    
    print(f"   2️⃣ Message déchiffré avec AES-GCM")
    print(f"\n   Correct : {plaintext == message} ✅")
    
    print(f"\n📊 Avantages du chiffrement hybride :")
    print(f"   ✅ Rapide : AES pour gros messages")
    print(f"   ✅ Sécurisé : RSA pour protection de clé")
    print(f"   ✅ Pas de limite de taille")
    print(f"   ✅ Utilisé dans : TLS 1.2, PGP, S/MIME")

hybrid_encryption_demo()

## Conclusion

**Points clés** :
- RSA basé sur difficulté de factorisation ($n = p \cdot q$)
- ❌ RSA textbook : Déterministe, homomorphe, vulnérable
- ✅ RSA-OAEP : Standard sécurisé pour chiffrement
- ✅ RSA-PSS : Standard sécurisé pour signatures
- ✅ Chiffrement hybride : RSA + AES pour messages longs

**Sécurité** :
- RSA-2048 : Sécurité ~112 bits (recommandé minimum)
- RSA-3072 : Sécurité ~128 bits (recommandé)
- RSA-4096 : Sécurité ~152 bits (très sécurisé, mais lent)

**Performance** :
- Chiffrement (clé publique) : Rapide
- Déchiffrement/Signature (clé privée) : ~10-100x plus lent
- Solution : Chiffrement hybride ou utiliser ECC

**En pratique** :
- TLS 1.3 : Préfère ECDHE + ECDSA (plus rapide)
- PGP/GPG : RSA + AES (hybride)
- SSH : RSA ou Ed25519 pour authentification